In [1]:
using POMDPs
using Random 
using Parameters
using StaticArrays
using Distributions
using MCTS
using POMDPSimulators

In [2]:
const Vec2 = SVector{2, Float64}
const Vec3 = SVector{3, Float64}

struct mystate
    uavPose::Vec3
    uavHeading::Float64 # radius
    targetPose::Vec2
end

struct myaction
    xy_speed::Float64 # m/s
    z_speed::Float64
    angle::Float64 # radius
end

@with_kw mutable struct UAVMDP <: MDP{mystate, myaction}

    target_velocity::Vec2 = SVector(0.2,0.2)
    target_std::Float64 = 0.1

    r_outScene::Float64 = -70
    r_action::Float64 = -0.1
    r_reach::Float64 = 100

    discount::Float64 = 0.95
    
    boundary::Int = 30
    landing_r::Int = 1

end

UAVMDP

In [3]:
function POMDPs.isterminal(p::UAVMDP,s::mystate)
    condition1 = (sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2) < p.landing_r)
    condition2 = abs(s.uavPose[1]) > p.boundary || abs(s.uavPose[2])>p.boundary || s.uavPose[3]<0 || s.uavPose[3]>p.boundary
    condition = condition1 || condition2
#     @show abs(s.uavPose[1]) > p.boundary
#     @show abs(s.uavPose[2]) > p.boundary
#     @show abs(s.uavPose[3]) > p.boundary
#     @show condition1
#     @show condition2
#     @show condition
    return condition
end

In [4]:
function POMDPs.generate_s(p::UAVMDP, s::mystate, a::myaction)
    
    # calculate target state
    target_move = p.target_velocity
    sensor_noise = rand(Normal(0, p.target_std), 2)
    curr_targ = s.targetPose + target_move + sensor_noise
    
    # calculate UAV state
    curr_angle = s.uavHeading + a.angle
    curr_angle = curr_angle%(2*pi)
    xy_dt_distance = a.xy_speed*SVector(cos(curr_angle), sin(curr_angle)) # careful
    z_dt_distance = a.z_speed
    xyz_dt_distance = SVector(xy_dt_distance[1], xy_dt_distance[2], z_dt_distance)
    curr_pos = s.uavPose + xyz_dt_distance
    return mystate(curr_pos, curr_angle, curr_targ)
end

In [5]:
function myreward(p::UAVMDP, s::mystate, a::myaction)
    
    distance_to_target = sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2)
    
    reward = p.r_action + 1/(distance_to_target+1)

    if abs(s.uavPose[1]) > p.boundary || abs(s.uavPose[2])>p.boundary || s.uavPose[3]<0 || s.uavPose[3]>p.boundary
        reward = reward + p.r_outScene
    end

    if distance_to_target < p.landing_r
        reward = reward + p.r_reach
    end
    return reward
end

myreward (generic function with 1 method)

In [6]:
function POMDPs.actions(p::UAVMDP)
    action_space = [myaction(i, j, k) for i=-2.0:0.5:2.0,j=-pi:pi/5:pi,k=-2:0.5:2]
    return action_space
end

In [7]:
function POMDPs.generate_sr(p::UAVMDP, s::mystate, a::myaction, rng::AbstractRNG)
    sp = generate_s(p, s, a)
    r = myreward(p, s, a)
    return sp, r
end

In [8]:
POMDPs.initialstate_distribution(p::UAVMDP) = mystate(SVector(1,1,1),0,SVector(0,0))
POMDPs.discount(p::UAVMDP) = p.discount
solver = MCTSSolver(n_iterations=100, depth=100, exploration_constant=15.0)
UAV = UAVMDP()

UAVMDP
  target_velocity: SArray{Tuple{2},Float64,1,2}
  target_std: Float64 0.1
  r_outScene: Float64 -70.0
  r_action: Float64 -0.1
  r_reach: Float64 100.0
  discount: Float64 0.95
  boundary: Int64 30
  landing_r: Int64 1


In [9]:
policy = solve(solver, UAV);

In [10]:
###trial
a = action(policy,mystate(SVector(10,10,10),0,SVector(0,0)))

myaction(-1.5, -3.141592653589793, -1.5)

In [11]:
uav_poses = [] # Vector of sequential uav poses
target_poses = []

0-element Array{Any,1}

In [12]:
i=0
terminate = false
reward_sum = 0

current_state = mystate(SVector(1,1,1),0,SVector(0,0)) ##initialize state

while i<1000 && !terminate
    current_action = action(policy,current_state)
    current_reward = myreward(UAV, current_state, current_action)
    current_state = generate_s(UAV, current_state, current_action)
    push!(uav_poses, current_state.uavPose)
    push!(target_poses, current_state.targetPose)
    terminate = isterminal(UAV,current_state)
    reward_sum = reward_sum + current_reward
    i=i+1

    if i%10==0
        @show reward_sum
        @show current_state
        @show current_action
        @show terminate
    end
end

reward_sum = 1.668180203508147
current_state = mystate([3.17898, 5.26101, 4.14159], -1.1504440784612413, [1.62895, 2.1072])
current_action = myaction(-1.0, 2.5132741228718345, -2.0)
terminate = false
reward_sum = 3.801857844539914
current_state = mystate([3.96864, 5.24708, 1.62832], -2.3008881569224826, [3.38247, 4.73232])
current_action = myaction(-2.0, -1.8849555921538759, -2.0)
terminate = false
reward_sum = 5.680780625265703
current_state = mystate([4.77622, 4.56009, 2.88496], -3.451332235383724, [5.66995, 6.41097])
current_action = myaction(-1.5, 0.0, -2.0)
terminate = false
reward_sum = 7.375578435369133
current_state = mystate([9.5788, 9.33057, 0.371681], -4.601776313844965, [7.90911, 8.87969])
current_action = myaction(1.5, -1.8849555921538759, -2.0)
terminate = false
reward_sum = 9.338570799362
current_state = mystate([8.78508, 11.0239, 2.88496], -5.7522203923062065, [9.41247, 10.7758])
current_action = myaction(-0.5, 1.8849555921538759, -2.0)
terminate = false
reward_sum = 10

In [13]:
using DelimitedFiles
writedlm( "uvapos.csv", uav_poses)
writedlm( "targetpos.csv", target_poses)